# Correlation Analysis with Neural Heatmap API

This notebook demonstrates advanced correlation analysis capabilities.

## What You'll Learn
- Download and analyze correlation matrices
- Filter correlations by layers or models
- Visualize multi-model comparisons
- Export correlation data for external analysis

In [ ]:
# Import required libraries
import asyncio
import json
from pathlib import Path

from neural_heatmap import NeuralHeatmapClient, connect, CorrelationMatrix
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for better visualizations
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

print("Libraries loaded successfully!")

## 1. Connect to Server

In [ ]:
async def connect_to_server():
    client = await connect("http://localhost:8080")
    if client.connected:
        print("✅ Connected to Neural Heatmap server")
    return client

client = await connect_to_server()

## 2. Download Full Correlation Matrix

Get the complete correlation matrix with all layers.

In [ ]:
async def download_full_matrix():
    # Get correlation matrix as DataFrame
    df = await client.get_correlation_matrix(as_dataframe=True)
    
    print(f"Correlation Matrix Shape: {df.shape}")
    print(f"\nLayers: {list(df.columns)}")
    print(f"\nCorrelation Matrix Summary:")
    print(df.describe())
    
    return df

correlation_df = await download_full_matrix()

## 3. Visualize Correlation Matrix with Heatmap

In [ ]:
def plot_correlation_heatmap(df, title="Neural Layer Correlation Matrix"):
    """Create a beautiful correlation heatmap"""
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Create heatmap with annotations
    sns.heatmap(
        df,
        annot=True,
        fmt='.2f',
        cmap='RdBu_r',
        center=0,
        vmin=-1,
        vmax=1,
        square=True,
        linewidths=0.5,
        cbar_kws={"shrink": 0.8, "label": "Correlation"},
        ax=ax
    )
    
    ax.set_title(title, fontsize=16, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()

plot_correlation_heatmap(correlation_df)

## 4. Analyze Strong Correlations

Find the strongest positive and negative correlations.

In [ ]:
def find_strong_correlations(df, threshold=0.7):
    """Find strong correlations above threshold"""
    # Get upper triangle (excluding diagonal)
    mask = np.triu(np.ones_like(df, dtype=bool), k=1)
    upper_tri = df.where(mask)
    
    # Find correlations above threshold
    strong_pos = []
    strong_neg = []
    
    for col in upper_tri.columns:
        for idx in upper_tri.index:
            val = upper_tri.loc[idx, col]
            if pd.notna(val):
                if val >= threshold:
                    strong_pos.append((idx, col, val))
                elif val <= -threshold:
                    strong_neg.append((idx, col, val))
    
    # Sort by correlation strength
    strong_pos.sort(key=lambda x: abs(x[2]), reverse=True)
    strong_neg.sort(key=lambda x: abs(x[2]), reverse=True)
    
    print(f"Strong Positive Correlations (≥{threshold}):")
    for layer1, layer2, corr in strong_pos[:10]:
        print(f"  {layer1} ↔ {layer2}: {corr:.3f}")
    
    print(f"\nStrong Negative Correlations (≤-{threshold}):")
    for layer1, layer2, corr in strong_neg[:10]:
        print(f"  {layer1} ↔ {layer2}: {corr:.3f}")
    
    return strong_pos, strong_neg

strong_pos, strong_neg = find_strong_correlations(correlation_df)

## 5. Filter by Specific Layers

Get correlations for a subset of layers.

In [ ]:
async def get_filtered_correlations(layer_ids):
    """Get correlation matrix for specific layers"""
    # Apply filter
    await client.set_filter(layer_ids=layer_ids)
    
    # Get filtered correlation matrix
    df = await client.get_correlation_matrix(as_dataframe=True)
    
    print(f"Filtered Correlation Matrix ({len(layer_ids)} layers):")
    print(df)
    
    return df

# Example: Get correlations for first 5 layers
all_layers = list(correlation_df.columns)
selected_layers = all_layers[:5]

filtered_df = await get_filtered_correlations(selected_layers)

## 6. Multi-Model Comparison

Compare correlations across multiple models.

In [ ]:
async def compare_models(model_ids):
    """Compare correlations across multiple models"""
    comparison = await client.compare_models(
        model_ids=model_ids,
        metrics=["correlation", "similarity"]
    )
    
    print(f"Multi-Model Comparison ({len(model_ids)} models):")
    print(json.dumps(comparison, indent=2))
    
    return comparison

# Example comparison
# comparison = await compare_models(["model1", "model2"])

## 7. Hierarchical Clustering of Layers

Use hierarchical clustering to group similar layers.

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist, squareform

def plot_hierarchical_clustering(df):
    """Create dendrogram showing layer similarity"""
    # Convert correlation to distance (1 - correlation)
    distance_matrix = 1 - df.abs()
    
    # Get condensed distance matrix
    dist_array = squareform(distance_matrix.values)
    
    # Perform hierarchical clustering
    linkage_matrix = linkage(dist_array, method='average')
    
    # Plot dendrogram
    fig, ax = plt.subplots(figsize=(12, 6))
    dendrogram(
        linkage_matrix,
        labels=df.columns,
        ax=ax,
        leaf_rotation=90,
        leaf_font_size=10
    )
    ax.set_title('Hierarchical Clustering of Neural Layers', fontsize=14)
    ax.set_xlabel('Layers')
    ax.set_ylabel('Distance')
    plt.tight_layout()
    plt.show()

plot_hierarchical_clustering(correlation_df)

## 8. Correlation Network Visualization

Create a network graph showing layer relationships.

In [ ]:
try:
    import networkx as nx
    
    def plot_correlation_network(df, threshold=0.5):
        """Create network graph of layer correlations"""
        # Create graph
        G = nx.Graph()
        
        # Add nodes
        for layer in df.columns:
            G.add_node(layer)
        
        # Add edges for strong correlations
        for i, col1 in enumerate(df.columns):
            for col2 in df.columns[i+1:]:
                corr = df.loc[col1, col2]
                if abs(corr) >= threshold:
                    # Edge width based on correlation strength
                    width = abs(corr) * 3
                    # Color based on positive/negative
                    color = 'green' if corr > 0 else 'red'
                    G.add_edge(col1, col2, width=width, color=color)
        
        # Draw graph
        fig, ax = plt.subplots(figsize=(14, 10))
        pos = nx.spring_layout(G, k=2, iterations=50)
        
        # Draw edges
        edges = G.edges(data=True)
        nx.draw_networkx_edges(
            G, pos,
            width=[e[2]['width'] for e in edges],
            edge_color=[e[2]['color'] for e in edges],
            alpha=0.6,
            ax=ax
        )
        
        # Draw nodes
        nx.draw_networkx_nodes(
            G, pos,
            node_color='lightblue',
            node_size=1000,
            alpha=0.8,
            ax=ax
        )
        
        # Draw labels
        nx.draw_networkx_labels(G, pos, font_size=10, ax=ax)
        
        ax.set_title(f'Neural Layer Correlation Network (threshold={threshold})', fontsize=14)
        ax.axis('off')
        plt.tight_layout()
        plt.show()
    
    plot_correlation_network(correlation_df, threshold=0.5)
    
except ImportError:
    print("NetworkX not installed. Install with: pip install networkx")

## 9. Export Correlation Data

Export correlation data in multiple formats.

In [ ]:
async def export_correlation_data():
    """Export correlation data in multiple formats"""
    export_dir = Path("./correlation_analysis")
    export_dir.mkdir(exist_ok=True)
    
    # Export to CSV
    await client.download_correlation_matrix(
        export_dir / "correlation_matrix.csv"
    )
    print("✅ Exported correlation_matrix.csv")
    
    # Export to JSON
    matrix = await client.get_correlation_matrix()
    with open(export_dir / "correlation_matrix.json", "w") as f:
        json.dump({
            "matrix": matrix.matrix,
            "labels": matrix.labels,
            "timestamp": matrix.timestamp,
            "metadata": matrix.metadata
        }, f, indent=2)
    print("✅ Exported correlation_matrix.json")
    
    # Export analysis results
    analysis = {
        "strong_positive": [(l1, l2, float(c)) for l1, l2, c in strong_pos],
        "strong_negative": [(l1, l2, float(c)) for l1, l2, c in strong_neg]
    }
    with open(export_dir / "correlation_analysis.json", "w") as f:
        json.dump(analysis, f, indent=2)
    print("✅ Exported correlation_analysis.json")

await export_correlation_data()

## 10. Cleanup

In [ ]:
async def cleanup():
    await client.clear_filter()
    await client.disconnect()
    print("✅ Disconnected from server")

await cleanup()

## Summary

In this notebook, you learned:
- How to download and visualize correlation matrices
- Techniques for analyzing strong correlations
- How to filter correlations by specific layers
- Multi-model comparison methods
- Hierarchical clustering of neural layers
- Network visualization of correlations
- Exporting correlation data for external analysis

## Next Steps
- `03_temporal_patterns.ipynb` - Analyze temporal patterns in neural activity
- `05_custom_workflows.ipynb` - Build custom analysis workflows